# Generate data based on BrainWeb images

This notebook contains most of the lines from the [BrainWeb notebook](BrainWeb.ipynb). Here, the images are cropped to a single slice, and only the motion case is considered. It could serve as inspiration for you on how to speed things up.

Authors: Richard Brown, Casper da Costa-Luis  
First version: 2nd of November 2019

CCP PETMR Synergistic Image Reconstruction Framework (SIRF)  
Copyright 2019  University College London  
Copyright 2019  King's College London  

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

SPDX-License-Identifier: Apache-2.0

In [ ]:
%matplotlib widget

# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('Synergistic', 'BrainWeb_single_slice')

import os
import brainweb
from brainweb import volshow
import numpy as np
from os import path
from tqdm.auto import tqdm
import logging
logging.basicConfig(level=logging.INFO)
import nibabel as nib
import sirf.STIR as pet
import matplotlib.pyplot as plt
import os
import sirf.Reg as reg
from math import cos, sin, pi, radians
from sirf.Utilities import examples_data_path
from sirf_exercises import exercises_data_path
import shutil
from scipy.ndimage.filters import gaussian_filter

## Get brainweb data (just single patient)

In [ ]:
fname, url= sorted(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(fname, url, ".")
data = brainweb.load_file(fname)

brainweb.seed(1337)

for f in tqdm([fname], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(
        f,
        petNoise=1, t1Noise=0.75, t2Noise=0.75,
        petSigma=1, t1Sigma=1, t2Sigma=1)
    vol_amyl = brainweb.get_mmr_fromfile(
        f,
        petNoise=1, t1Noise=0.75, t2Noise=0.75,
        petSigma=1, t1Sigma=1, t2Sigma=1,
        PetClass=brainweb.Amyloid)

FDG_arr  = vol['PET']
amyl_arr = vol_amyl['PET']
uMap_arr = vol['uMap']
T1_arr   = vol['T1']
T2_arr   = vol['T2']

# Take centre slice to make 2d
slice = FDG_arr.shape[0]//2
FDG_arr  = FDG_arr[slice,:,:]
amyl_arr = amyl_arr[slice,:,:]
uMap_arr = uMap_arr[slice,:,:]
T1_arr   = T1_arr[slice,:,:]
T2_arr   = T2_arr[slice,:,:]

## Display it

In [ ]:
def subplot_(idx,vol,title,clims=None,cmap="viridis"):
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")

plt.figure();

subplot_([2,3,1],FDG_arr [100:-100, 100:-100],'FDG'    ,cmap="hot")
subplot_([2,3,2],amyl_arr[100:-100, 100:-100],'Amyloid',cmap="hot")
subplot_([2,3,3],uMap_arr[100:-100, 100:-100],'uMap'   ,cmap="bone")
subplot_([2,3,4],T1_arr  [100:-100, 100:-100],'T1'     ,cmap="Greys_r")
subplot_([2,3,5],T2_arr  [100:-100, 100:-100],'T2'     ,cmap="Greys_r")

## Crop images and save

Here's what's going on in this cell:

1. The data from brainweb is (127,344,344), but we want it to be (1,150,150). We've already cropped in the z-direction to (1,344,344). For x-y, just keep the middle sections of the image in the x-y plane.
2. Crop the image to reduce it to (1,150,150).
3. Save the image to file, both interfile and nifti

N.B.: This requires you to have a version of SIRF > v2.1.0. See the cell at the bottom of this notebook if you have an older version of SIRF.

In [ ]:
# We'll need a template sinogram
mmr_template_sino_single_slice = examples_data_path('PET') + '/mMR/mMR_template_single_slice.hs'
templ_sino = pet.AcquisitionData(mmr_template_sino_single_slice)

def crop_and_save(templ_sino, im_array, fname):
    # Crop from (1,344,344) to (1,150,150) and save to file
    im_array = im_array[97:97+150,97:97+150]
    im_array = np.expand_dims(im_array, axis = 0)
    im = pet.ImageData(templ_sino)
    dim=(1,150,150)
    voxel_size=im.voxel_sizes()
    im.initialise(dim,voxel_size)
    im.fill(im_array)
    im.write(fname)
    im_nii = reg.ImageData(im)
    im_nii.write(fname)
    return im
    
FDG  = crop_and_save(templ_sino, FDG_arr,  "FDG"    )
amyl = crop_and_save(templ_sino, amyl_arr, "Amyloid")
uMap = crop_and_save(templ_sino, uMap_arr, "uMap"   )
T1   = crop_and_save(templ_sino, T1_arr,   "T1"     )
T2   = crop_and_save(templ_sino, T2_arr,   "T2"     )

In [ ]:
plt.figure();
subplot_([2,3,1],np.squeeze(FDG.as_array()),  'FDG'    ,cmap="hot")
subplot_([2,3,2],np.squeeze(amyl.as_array()), 'Amyloid',cmap="hot")
subplot_([2,3,3],np.squeeze(uMap.as_array()), 'uMap'   ,cmap="bone")
subplot_([2,3,4],np.squeeze(T1.as_array()),   'T1'     ,cmap="Greys_r")
subplot_([2,3,5],np.squeeze(T2.as_array()),   'T2'     ,cmap="Greys_r")

# Add motion

Resample the image using SIRF's wrapper around NiftyReg. 

As with the crop, moving an image around affects its offset in STIR, which currently causes problems. So again, we recentre the image after the resample.

In [ ]:
def add_misalignment(transformation_matrix,image):

    # Resample
    resampler = reg.NiftyResample()
    resampler.set_interpolation_type_to_cubic_spline()
    resampler.set_reference_image(image)
    resampler.set_floating_image(image)
    resampler.set_padding_value(0)
    resampler.add_transformation(transformation_matrix)
    resampler.process()

    # Save to file
    resampled = resampler.get_output()

    # Remove all offset info (avoids problems in STIR)
    misaligned_image = resampled.move_to_scanner_centre(templ_sino)
    return misaligned_image

### Create the transformation matrix

The rotation matrix we'll use here is a rotation of 30 degrees about one of the axes, and a translation of 20 and -10 mm in the x- and y-directions, respectively.

In [ ]:
input_ims = [FDG, amyl, uMap, T1, T2]
FDGs = []
amyls = []
uMaps = []
T1s = []
T2s = []
num_motion_states = 4
for i in range(num_motion_states):
    
    # Get some motion
    if i == 0:
        [r,t_x,t_y] = [0., 0., 0.]
    elif i == 1: 
        [r,t_x,t_y] = [10., -10., 0.]
    elif i == 2: 
        [r,t_x,t_y] = [20., -5., 5.]
    elif i == 3: 
        [r,t_x,t_y] = [-10., 10., 5.]
    else:
        raise AssertionError('need more motion')
        
    r = radians(r)
    tm = reg.AffineTransformation(np.array(\
        [[ cos(r), sin(r), 0, t_x], \
         [-sin(r), cos(r), 0, t_y], \
         [      0,      0, 1, 0  ], \
         [      0,      0, 0, 1  ]]))
    
    # Apply motion to all FDG, uMap, T1, etc.
    resampled_ims = []
    for j in range(len(input_ims)):
        im = input_ims[j]
        # Resample
        resampler = reg.NiftyResample()
        resampler.set_interpolation_type_to_cubic_spline()
        resampler.set_reference_image(im)
        resampler.set_floating_image(im)
        resampler.set_padding_value(0)
        resampler.add_transformation(tm)
        resampled = resampler.forward(im)
        resampled.move_to_scanner_centre(templ_sino)
        
        if j==0:
            fname = 'FDG'
            FDGs.append(resampled)
        elif j==1:
            fname = 'amyl'
            amyls.append(resampled)
        elif j==2:
            fname = 'uMap'
            uMaps.append(resampled)
        elif j==3:
            fname = 'T1'
            T1s.append(resampled)
        elif j==4:
            fname = 'T2'
            T2s.append(resampled)
            
        reg.ImageData(resampled).write(fname + '_mf' + str(i))
        tm.write('fwd_tm_mf' + str(i))
        tm.get_inverse().write('inv_tm_mf' + str(i))

In [ ]:
# Display
plt.figure();
for i in range(num_motion_states):
    subplot_([2,2,i+1], np.squeeze(FDGs[i].as_array()), 'Motion state ' + str(i), clims=[0, 160])

## Forward project

Forward project both the FDG and amyloid images both with and without Poisson noise.

In [ ]:
def get_acquisition_model(uMap, templ_sino):

    #%% create acquisition model
    am = pet.AcquisitionModelUsingRayTracingMatrix()
    am.set_num_tangential_LORs(5)

    # Set up sensitivity due to attenuation
    asm_attn = pet.AcquisitionSensitivityModel(uMap, am)
    asm_attn.set_up(templ_sino)
    bin_eff = pet.AcquisitionData(templ_sino)
    bin_eff.fill(1.0)
    print('applying attenuation (please wait, may take a while)...')
    asm_attn.unnormalise(bin_eff)
    asm_attn = pet.AcquisitionSensitivityModel(bin_eff)

    am.set_acquisition_sensitivity(asm_attn)

    am.set_up(templ_sino,uMap);
    return am

In [ ]:
# Function for adding noise
def add_noise(proj_data,noise_factor = 1):
    proj_data_arr = proj_data.as_array() / noise_factor
    # Data should be >=0 anyway, but add abs just to be safe
    proj_data_arr = np.abs(proj_data_arr)
    noisy_proj_data_arr = np.random.poisson(proj_data_arr).astype('float32');
    noisy_proj_data = proj_data.clone()
    noisy_proj_data.fill(noisy_proj_data_arr);
    return noisy_proj_data

In [ ]:
FDG_sinos = []
FDG_sinos_noisy = []

for i in range(num_motion_states):
    am = get_acquisition_model(uMaps[i], templ_sino)
    FDG_sino = am.forward(FDGs[i])
    noisy_FDG_sino = add_noise(FDG_sino,1000)
    FDG_sino.write('sino_FDG_mf' + str(i))
    noisy_FDG_sino.write('sino_FDG_noisy_mf' + str(i))
    FDG_sinos.append(FDG_sino)
    FDG_sinos_noisy.append(noisy_FDG_sino)

In [ ]:
FDG_sinos[i].as_array().shape
plt.figure();
for i in range(num_motion_states):
    subplot_([num_motion_states,2,i*2+1],   FDG_sinos[i].as_array()[0,0,:,:], 'FDG sino, mf' + str(i))
    subplot_([num_motion_states,2,i*2+2], FDG_sinos_noisy[i].as_array()[0,0,:,:], 'FDG sino (noisy), mf' + str(i))